Okay, let's outline a Python script to migrate MariaDB tables to AWS Redshift Serverless. This is a relatively complex task, so we'll break it down and provide a comprehensive solution with explanations.

**Key Challenges and Considerations:**

*   **Data Transfer:** We need an efficient way to extract data from MariaDB and load it into Redshift.
*   **Data Type Mapping:** MariaDB and Redshift have different data type systems. Careful mapping is crucial to avoid data loss or errors.
*   **Schema Creation:** We'll need to create tables in Redshift that match the MariaDB schema.
*   **Performance:** For large tables, we need to consider performance and optimize the data transfer process.
*   **Credentials Management:** We'll need secure ways to handle credentials for both databases.
*   **Error Handling:** Robust error handling is important for troubleshooting issues.
*   **Scalability:** This script should be able to handle multiple tables.

**Required Libraries:**

You'll need to install these libraries using `pip install`:

```bash
pip install mysql-connector-python psycopg2-binary boto3 pandas sqlalchemy
```

**Python Script:**

```python
import mysql.connector
import psycopg2
import boto3
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from tqdm import tqdm
import os
import logging
from dotenv import load_dotenv

load_dotenv()


# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database Configuration (Load from environment)
MARIADB_HOST = os.getenv("MARIADB_HOST")
MARIADB_USER = os.getenv("MARIADB_USER")
MARIADB_PASSWORD = os.getenv("MARIADB_PASSWORD")
MARIADB_DATABASE = os.getenv("MARIADB_DATABASE")
REDSHIFT_HOST = os.getenv("REDSHIFT_HOST")
REDSHIFT_USER = os.getenv("REDSHIFT_USER")
REDSHIFT_PASSWORD = os.getenv("REDSHIFT_PASSWORD")
REDSHIFT_DATABASE = os.getenv("REDSHIFT_DATABASE")
REDSHIFT_PORT = os.getenv("REDSHIFT_PORT")

# Other Settings
CHUNK_SIZE = 10000  # Number of rows to fetch and insert at a time

# Data type mapping
DATA_TYPE_MAP = {
    "tinyint": "SMALLINT",
    "smallint": "SMALLINT",
    "mediumint": "INTEGER",
    "int": "INTEGER",
    "integer": "INTEGER",
    "bigint": "BIGINT",
    "float": "FLOAT",
    "double": "DOUBLE PRECISION",
    "decimal": "DECIMAL",
    "date": "DATE",
    "datetime": "TIMESTAMP",
    "timestamp": "TIMESTAMP",
    "time": "TIME",
    "year": "SMALLINT",
    "char": "CHAR",
    "varchar": "VARCHAR",
    "text": "TEXT",
    "mediumtext": "TEXT",
    "longtext": "TEXT",
    "blob": "BYTEA",
    "mediumblob": "BYTEA",
    "longblob": "BYTEA",
    "enum": "VARCHAR",  # Handle enum as VARCHAR
    "set": "VARCHAR",   # Handle set as VARCHAR
}



def get_mariadb_connection():
    """Creates and returns a MariaDB connection."""
    try:
        conn = mysql.connector.connect(
            host=MARIADB_HOST,
            user=MARIADB_USER,
            password=MARIADB_PASSWORD,
            database=MARIADB_DATABASE,
        )
        conn.autocommit = True
        return conn
    except mysql.connector.Error as err:
        logging.error(f"Error connecting to MariaDB: {err}")
        raise


def get_redshift_engine():
    """Creates and returns a SQLAlchemy engine for Redshift."""
    try:
      
        redshift_uri = f"postgresql+psycopg2://{REDSHIFT_USER}:{REDSHIFT_PASSWORD}@{REDSHIFT_HOST}:{REDSHIFT_PORT}/{REDSHIFT_DATABASE}"
        engine = create_engine(redshift_uri)
        return engine
    except SQLAlchemyError as e:
        logging.error(f"Error connecting to Redshift: {e}")
        raise


def get_mariadb_tables(conn):
    """Fetches a list of all tables in the MariaDB database."""
    cursor = conn.cursor()
    cursor.execute("SHOW TABLES")
    tables = [table[0] for table in cursor.fetchall()]
    cursor.close()
    return tables


def get_mariadb_table_schema(conn, table_name):
    """Fetches the schema of a table from MariaDB."""
    cursor = conn.cursor()
    cursor.execute(f"SHOW COLUMNS FROM `{table_name}`")
    columns = cursor.fetchall()
    cursor.close()

    schema = []
    for column in columns:
      col_name = column[0]
      col_type = column[1].split('(')[0]
      col_nullable = column[2] == 'YES'
      col_default = column[4]
      
      redshift_type = DATA_TYPE_MAP.get(col_type, "VARCHAR") # Default to VARCHAR if not found
      
      schema.append({
          'name': col_name,
          'type': redshift_type,
          'nullable': col_nullable,
          'default': col_default,
      })

    return schema

def create_redshift_table(redshift_engine, table_name, schema):
    """Creates a table in Redshift based on the provided schema."""
    
    columns_definition = []
    for column in schema:
      col_name = column['name']
      col_type = column['type']
      col_nullable = 'NULL' if column['nullable'] else 'NOT NULL'
      col_default = f"DEFAULT '{column['default']}'" if column['default'] else ''
      
      columns_definition.append(f'"{col_name}" {col_type} {col_nullable} {col_default}'.strip())
      
    table_definition = ','.join(columns_definition)
    create_table_sql = f'CREATE TABLE IF NOT EXISTS "{table_name}" ({table_definition});'

    try:
      with redshift_engine.connect() as connection:
        connection.execute(text(create_table_sql))
        logging.info(f'Table "{table_name}" created successfully in Redshift')
    except SQLAlchemyError as e:
        logging.error(f'Error creating table "{table_name}" in Redshift: {e}')
        raise

def migrate_table_data(mariadb_conn, redshift_engine, table_name):
    """Migrates data from a MariaDB table to Redshift."""
    logging.info(f'Migrating data for table "{table_name}"')

    try:
        mariadb_cursor = mariadb_conn.cursor(dictionary=True) #fetch rows as dict
        mariadb_cursor.execute(f"SELECT * FROM `{table_name}`")

        while True:
          rows = mariadb_cursor.fetchmany(CHUNK_SIZE)
          if not rows:
              break

          df = pd.DataFrame(rows)
            
          # Convert columns with datetime object to string 
          for col in df.columns:
              if pd.api.types.is_datetime64_any_dtype(df[col]):
                  df[col] = df[col].astype(str)
          
          df.to_sql(
                table_name,
                redshift_engine,
                if_exists="append",
                index=False,
                method='multi',
                chunksize=CHUNK_SIZE,  
            )
          
          logging.info(f'Inserted {len(rows)} rows into table "{table_name}"')

        mariadb_cursor.close()

    except Exception as e:
      logging.error(f'Error migrating data for table "{table_name}": {e}')
      raise

def main():
    """Main function to orchestrate the database migration."""
    try:
        mariadb_conn = get_mariadb_connection()
        redshift_engine = get_redshift_engine()
        
        mariadb_tables = get_mariadb_tables(mariadb_conn)
        
        logging.info(f"Found {len(mariadb_tables)} tables in MariaDB: {mariadb_tables}")
        
        for table_name in mariadb_tables:
          logging.info(f"Processing table: {table_name}")
          
          schema = get_mariadb_table_schema(mariadb_conn, table_name)
          create_redshift_table(redshift_engine, table_name, schema)
          migrate_table_data(mariadb_conn, redshift_engine, table_name)
          
          logging.info(f"Migration finished for {table_name}")
        
        logging.info('Database migration completed successfully')

    except Exception as e:
        logging.error(f"Migration failed: {e}")

    finally:
        if 'mariadb_conn' in locals() and mariadb_conn.is_connected():
            mariadb_conn.close()
        if 'redshift_engine' in locals() and redshift_engine:
            redshift_engine.dispose()

if __name__ == "__main__":
    main()
```

**How to Use:**

1.  **Set up Environment Variables:** Create a `.env` file in the same directory as your Python script and fill in your database details:
    ```
    MARIADB_HOST=your_mariadb_host
    MARIADB_USER=your_mariadb_user
    MARIADB_PASSWORD=your_mariadb_password
    MARIADB_DATABASE=your_mariadb_database

    REDSHIFT_HOST=your_redshift_host
    REDSHIFT_USER=your_redshift_user
    REDSHIFT_PASSWORD=your_redshift_password
    REDSHIFT_DATABASE=your_redshift_database
    REDSHIFT_PORT=5439  # Default Redshift port
    ```
2.  **Install Dependencies:** `pip install mysql-connector-python psycopg2-binary boto3 pandas sqlalchemy python-dotenv tqdm`
3.  **Run the script:** `python your_script_name.py`

**Explanation:**

1.  **Configuration:**
    *   Loads database credentials and settings from environment variables using `dotenv`.
    *   Defines global constants like `CHUNK_SIZE`.
    *   Sets up basic logging for progress and errors.
    *   Defines data type mapping from MariaDB to Redshift.

2.  **Connection Functions:**
    *   `get_mariadb_connection()`: Creates a connection to your MariaDB database using `mysql.connector`.
    *   `get_redshift_engine()`: Creates a SQLAlchemy engine for Redshift using `psycopg2`.

3.  **Schema Handling Functions:**
    *   `get_mariadb_tables()`: Gets a list of tables in MariaDB using `SHOW TABLES`.
    *   `get_mariadb_table_schema()`: Gets the column definitions (name, data type, nullability, etc.) using `SHOW COLUMNS`. It uses the `DATA_TYPE_MAP` to translate MariaDB types to appropriate Redshift types.
    *   `create_redshift_table()`: Creates tables in Redshift using a constructed `CREATE TABLE` query with SQLAlchemy.

4.  **Data Migration Functions:**
    *   `migrate_table_data()`:
        *   Fetches data in batches using `fetchmany` with specified `CHUNK_SIZE`.
        *   Uses `pandas` to convert the data into dataframes and convert datetime objects to string.
        *   Uses `pandas.to_sql()` to efficiently insert data into Redshift.

5.  **Main Function:**
    *   Orchestrates all steps: gets connections, lists tables, processes each table: gets schema, creates the table, and migrates the data.
    *   Handles errors with a `try/except` block.
    *   Closes connections in a `finally` block to release resources.

**Important Notes:**

*   **Large Databases:** For very large tables, you might need to explore more advanced techniques:
    *   **AWS DMS:** Consider using AWS Database Migration Service (DMS) for a more robust and feature-rich approach.
    *   **S3 Staging:** You could extract data from MariaDB into CSV files, upload them to S3, and then use Redshift's `COPY` command to ingest them (faster for large datasets).
*   **Data Type Mapping:** The provided `DATA_TYPE_MAP` is a starting point. Carefully review your database schema and adjust it as needed for full compatibility with Redshift.
*   **Error Handling:** The script includes basic error handling. You may need to add more fine-grained checks and logging based on your requirements.
*   **Performance Tuning:** Experiment with the `CHUNK_SIZE` and consider using larger instance types for Redshift to optimize migration speed.
*   **Permissions:** Make sure that your AWS credentials used by `boto3` have the necessary permissions to access Redshift and create tables. Your user should also have permissions on MariaDB to perform the queries.
*   **Production Use:** This script should be thoroughly tested in a development or staging environment before running it in production.
*   **Complex Schema:** For extremely complex schemas with foreign keys, constraints, etc. consider using an ETL tool or other more feature-rich solutions.

This script provides a solid foundation for migrating data from MariaDB to Redshift Serverless. Please adjust and enhance it based on your specific needs and data requirements. Remember to always back up your data before performing any major migrations.
